<center>
    <h2>The Cleaning and Wrangling of the Yelp Dataset<h2>
</center>

<b>Introduction:</b>
<br></br>
Yelp is company designed around the collection of reviews for the users of its application to be able to make informed decisions about their consumer needs. As a part of an academic data challenge, Yelp has provided a subset of our businesses, reviews, and user data. The datasets provided were in JSON format and included over 1.4 million business attributes, 188,593 businesses, and 5,996,996 reviews. For the project, we wanted to examine the restaurant reviews with sentiment analysis. Our analysis is broken in to two scripts, a data preprocessing script and our data analysis script. In this script, we preforming cleaning of our data and merge our two large Yelp data sets together. Lastly, we will write to csv in order for our analysis to process more quickly.

In [2]:
import json
import pandas as pd
import re
from itertools import islice


<b>Yelp Business Data</b>
<br></br>
Data Source: Yelp.com Dataset Challenge (also available on Kaggle)
<br></br>
File Type: JSON
<br></br>
File Size: 1.1 GB
<br></br>
Modules Used: pandas


In [8]:
#import Business data as pandas dataframe
yelp_business = pd.read_json('yelp_business.json', orient = 'columns', lines = True)
yelp_business.head(3)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",MontrÃ©al,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC


For the purposes and scope of our project, we identified columns that were not needed and therefore dropped the following attributes to reduce data size:
<br></br>
- Business address
- Business attributes
- Business hours
- Is_open
- Latitude
- Longitude
- Neighborhood


In [9]:
#remove columns
yelp_business.drop(['address', 'attributes', 'hours', 'is_open', 'latitude', 'longitude', 'neighborhood'], axis=1, inplace=True)

We noticed odd postal codes in the postal_code category. Upon further investigation there were international businesses included in this dataset. We wanted to stick with US/domestic businesses only, and removed all rows containing a postal code that was not a 5 digit number.

Next, we tackled the problem of the sheer number of categories and businesses. There were about 188,000 businesses and over 1,200 different categories. To narrow down the scope of our project, we decided to focus only on restaurants. We eliminated any business that did not contain the general category ‘restaurant’ in its long list of possible categories.

In [10]:
#extract only Restaurants in category and 5 digit postal codes
yelp_business_cleaned = yelp_business[yelp_business['categories'].str.contains('Restaurants') == True]
yelp_business_cleaned = yelp_business_cleaned[yelp_business_cleaned['postal_code'].str.contains('^(\d{5})$') == True]

C:\Users\kyhwa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#transform categories out of multi value
test_cat = yelp_business_cleaned.set_index('business_id').categories.str.split(', ', expand=True).stack()

In [12]:
test_cat.head()

business_id              
AjEbIBw6ZFfln7ePHha9PA  0     Chicken Wings
                        1           Burgers
                        2          Caterers
                        3    Street Vendors
                        4          Barbeque
dtype: object

In [13]:
#reset index
test_cat = test_cat.reset_index()

In [14]:
test_cat.head()

,business_id,level_1,0
0,AjEbIBw6ZFfln7ePHha9PA,0,Chicken Wings
1,AjEbIBw6ZFfln7ePHha9PA,1,Burgers
2,AjEbIBw6ZFfln7ePHha9PA,2,Caterers
3,AjEbIBw6ZFfln7ePHha9PA,3,Street Vendors
4,AjEbIBw6ZFfln7ePHha9PA,4,Barbeque


In [15]:
#rename columns
test_cat = test_cat.rename(index=str, columns={0: "category"})
test_cat = test_cat.rename(index=str, columns={'level_1': "instance"})
test_cat.head()

,business_id,instance,category
0,AjEbIBw6ZFfln7ePHha9PA,0,Chicken Wings
1,AjEbIBw6ZFfln7ePHha9PA,1,Burgers
2,AjEbIBw6ZFfln7ePHha9PA,2,Caterers
3,AjEbIBw6ZFfln7ePHha9PA,3,Street Vendors
4,AjEbIBw6ZFfln7ePHha9PA,4,Barbeque


In [17]:
#merge transformed categories to original dataframe
businesses_final = pd.merge(yelp_business_cleaned, test_cat, on='business_id', how = 'inner')

In [18]:
#rearrange columns
businesses_final = businesses_final[['business_id', 'name', 'category', 'city', 'state', 'postal_code', 'review_count','stars','instance','categories']]
businesses_final.head()

,business_id,name,category,city,state,postal_code,review_count,stars,instance,categories
0,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,Chicken Wings,Henderson,NV,89002,3,4.5,0,"Chicken Wings, Burgers, Caterers, Street Vendo..."
1,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,Burgers,Henderson,NV,89002,3,4.5,1,"Chicken Wings, Burgers, Caterers, Street Vendo..."
2,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,Caterers,Henderson,NV,89002,3,4.5,2,"Chicken Wings, Burgers, Caterers, Street Vendo..."
3,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,Street Vendors,Henderson,NV,89002,3,4.5,3,"Chicken Wings, Burgers, Caterers, Street Vendo..."
4,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,Barbeque,Henderson,NV,89002,3,4.5,4,"Chicken Wings, Burgers, Caterers, Street Vendo..."


We then take our unique business ids that are now just businesses categorized as businesses to use a filter for the review dataset.

In [19]:
businesses_final_distinct = list(businesses_final['business_id'].unique())
businesses_final_distinct
len(businesses_final_distinct)

34332

<b> Yelp Review Data </b>
<br></br>
Data Source: Yelp.com Dataset Challenge (also available on Kaggle)
<br></br>
File Type: JSON
<br></br>
File Size: 4.6 GB
<br></br>
Modules Used: json, pandas, re


The Yelp Review data was also in JSON format and included nearly six million reviews. This set was difficult to read in due to its enormous size. Using the subsetted Business dataset that was subsetted into restaurants, we were able to use the business ids to filter the review dataset as we brought it in. This was a very time consuming process.

In [ ]:
yr_file = 'review_150k.json'
#creates list of key value pairs
yelp_reviews = []
for line in open(yr_file, encoding="utf8"):
    line = json.loads(line)
    is_restaurant = line['business_id'] in list(businesses_final_distinct)
    if not is_restaurant: 
        continue
    yelp_reviews.append(line)

# View type of yelp_reviews
print(type(yelp_reviews))
print(len(yelp_reviews))
print(type(yelp_reviews[0]))

In [9]:
#Creating a pandas data frame
yelp_reviews_df = pd.DataFrame.from_records(yelp_reviews)

When the JSON Yelp reviews were read in they were in the form of a list of dictionaries. We transformed the data from a list to a data frame in the above code chunk, but the dictionaries were still embedded with in our our data frame. To solve this problem we had to write a function, as seen below. The function strip_dict_values returns the value from the dictionary key value pair. Then with an apply function, we can apply our function to all the columns in our dataframe.  

In [11]:
def strip_dict_values(x):
    if isinstance(x,dict):
        v = list(x.values())
        return v[0]
    else:
        return x

cols_with_dict = list(yelp_reviews_df)

for col in cols_with_dict:
 yelp_reviews_df[col] = yelp_reviews_df[col].apply(strip_dict_values)

In [12]:
yelp_reviews_df

,_id,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,5bfb30c40d7c4d55f05b4aba,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,5bfb30c40d7c4d55f05b4abb,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,5bfb30c40d7c4d55f05b4abc,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,5bfb30c40d7c4d55f05b4abd,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,5bfb30c40d7c4d55f05b4abe,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g
5,5bfb30c40d7c4d55f05b4ac0,AxeQEz3-s9_1TyIo-G7UQw,0,2011-10-10,0,brokEno2n7s4vrwmmUdr9w,5,"If you like Thai food, you have to try the ori...",1,msQe1u7Z_XuqjGoqhB0J5g
6,5bfb30c40d7c4d55f05b4ac1,zdE82PiD6wquvjYLyhOJNA,0,2012-04-18,1,kUZWBVZvhWuC8TWUg5AYyA,5,AMAZING!!!\n\n I was referred here by a friend...,0,msQe1u7Z_XuqjGoqhB0J5g
7,5bfb30c40d7c4d55f05b4ac2,EAwh1OmG6t6p3nRaZOW_AA,0,2011-02-25,0,wcqt0III88LEcm19IxFFyA,4,Ribs = amazing\n2 hour wait time= not so amazi...,0,msQe1u7Z_XuqjGoqhB0J5g
8,5bfb30c40d7c4d55f05b4ac3,atVh8viqTj-sqDJ35tAYVg,1,2012-11-09,2,LWUtqzNthMM3vpWZIFBlPw,2,"Food is pretty good, not gonna lie. BUT you ha...",1,msQe1u7Z_XuqjGoqhB0J5g
9,5bfb30c40d7c4d55f05b4ac5,UBv8heCQR0RPnUQG0zkXIQ,0,2016-09-23,0,HkYqGb0Gplmmk-xlHTRBoA,1,The score should be negative. Its HORRIBLE. Th...,0,NhOc64RsrTT1Dls50yYW8g



After the review dataset was cleaned, it was ready to be joined with the business data set. We were able to accomplish this through a merge using the business_id. The end result of our merge was a dataframe with 17 columns and 69, 429 rows.  A glimpse of the data set can be seen 


In [16]:
yelp_restaurant_final = pd.merge(yelp_business_cleaned, yelp_reviews_df, on='business_id', how = 'inner')

yelp_restaurant_final

,business_id,categories,city,name,postal_code,review_count,stars_x,state,_id,cool,date,funny,review_id,stars_y,text,useful,user_id
0,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30c90d7c4d55f05c518e,0,2015-05-11,0,lM_XM7e1nD7d7NJ815inuA,4,Cozy neighborhood sports bar w good burgers. L...,2,UfUFjbwLpYCeJrWUWdMYVA
1,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30c90d7c4d55f05c592c,0,2010-08-14,0,ACW_G1G0PG0GNyGUPfI3UA,4,The bad: the bar closes at 10pm. It seems lik...,0,VWDL0VgQ2ivpN3oYhL_WsA
2,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30ca0d7c4d55f05c6dc2,1,2014-07-29,0,KKJa4pRGwq8eO6YCdhjNoA,5,"Good food, good vibes, good service. All adds ...",1,SoL4ToJdvxWpGGzxYVA86A
3,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30ca0d7c4d55f05c6f0b,0,2015-11-08,0,zSa_s88VlxYzeiHHTlK6xw,5,My wife and I were referred to this place by a...,0,JIj2V2YaM5_8jVtnkLSZ5g
4,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30cc0d7c4d55f05cd602,3,2009-09-24,5,1wYTbfyQ0bfM31nM9c3hsA,3,My boyfriend likes burgers...I don't. Not real...,4,BoJQWDp2a9sxUMrlJBTfRA
5,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30cc0d7c4d55f05cf63f,0,2013-09-29,1,ylh2-hOBdV7uzdinX3pbfw,5,Border Burger is great. Onion rings are awesom...,0,0CuIwjdRrcOsUrO-2gE-9A
6,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30cd0d7c4d55f05d03ed,0,2015-01-23,0,_yf_sxbOp63YVc9Mdys_cw,1,Unimpressed. The place was pretty run down. T...,0,7BKHMOG-E0fW2VGWajLAjA
7,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30cd0d7c4d55f05d1621,0,2013-07-06,0,_QF1GZE6f9-eWb2GGtcgUw,4,Great flame broiled burgers. Used to be my fa...,0,FrLvmygqsfjtHGRx4-poaA
8,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30cd0d7c4d55f05d281c,0,2011-08-09,0,A98EBKgfyxrsnjzosbXM2A,3,I had a Garden Burger and fries. It was prett...,0,3rAF4TU6nIjBztE7Pq2lPg
9,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,Original Hamburger Works,85007,277,4.0,AZ,5bfb30cd0d7c4d55f05d2979,1,2014-09-06,0,XDCl4tqaQMJrJ0z07bU_Qg,4,Came here for the first time with my 19-year o...,1,RkEd2r0YcEVSsaiO-ZD-OQ


To save on time from loading  the larger datasets while performing analysis, we lastly wrote our cleaned and merged data frame to CSV. 

In [18]:
yelp_restaurant_final.to_csv("Yelp_restaurant_reviews.csv", header = True)

Now with our new much smaller csv written, we can begin the analysis of the Yelp Restaurant Reviews. Please see our next Juypter Notebook file entiled Hwang_Crockett_finalproject_sentimentanalysis.ipynb.

# basic exploration of data

In [20]:
#category basic statistics
businesses_final['category'].describe()

count          141880
unique            649
top       Restaurants
freq            34332
Name: category, dtype: object

In [21]:
#restaurants id basic statistics
businesses_final['business_id'].describe()

count                     141880
unique                     34332
top       IjsLANGkmAqCsF6-zgIA8w
freq                          37
Name: business_id, dtype: object

In [22]:
#stars rating basic statistics
businesses_final['stars'].describe()

count    141880.000000
mean          3.498210
std           0.811818
min           1.000000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: stars, dtype: float64

In [23]:
#median stars rating across all restaurants
print(businesses_final['stars'].median())
print(' ')
print('note: restaurants rating has a normal distribution since the mean and median are almost the same.')

3.5
 
note: restaurants rating has a normal distribution since the mean and median are almost the same.


In [24]:
#unique businesses by category
unique = businesses_final.groupby('category')['business_id'].nunique().reset_index()

In [25]:
#sort and view by most dominant categories
unique_sorted = unique.sort_values('business_id', ascending = False).reset_index()

In [26]:
#new df for categories and unique business count
unique_sorted = unique_sorted[['category', 'business_id']]
unique_sorted = unique_sorted.rename(index=str, columns={'business_id': "business_count"})
unique_sorted

,category,business_count
0,Restaurants,34332
1,Food,7196
2,American (Traditional),5467
3,Nightlife,5356
4,Bars,5169
5,Fast Food,5134
6,Sandwiches,5009
7,Pizza,4571
8,American (New),3909
9,Mexican,3789


In [27]:
#average star rating per category (unique businesses only aka instance = 0)
stars = businesses_final[businesses_final.instance == 0].groupby('category')['stars'].mean().reset_index()

In [28]:
#combine unique sorted categories and avg star ratings
categories = pd.merge(unique_sorted, stars, on='category', how = 'inner')
categories = categories.rename(index=str, columns={'stars': "avg_stars"})

In [29]:
#sort by highest rated categories - not very insightful
categories.sort_values('avg_stars', ascending = False)

,category,business_count,avg_stars
358,Historical Tours,1,5.000000
298,Bike Rentals,3,5.000000
228,Tobacco Shops,11,5.000000
269,Themed Cafes,5,5.000000
272,DJs,5,5.000000
280,Gyms,5,5.000000
281,Grilling Equipment,5,5.000000
285,Car Wash,4,5.000000
297,Makeup Artists,3,5.000000
305,Outdoor Furniture Stores,3,5.000000


In [30]:
#highest rated categories if business count > 20
categories[(categories.business_count > 20)].sort_values('avg_stars', ascending = False)

,category,business_count,avg_stars
181,Personal Chefs,24,4.833333
179,Professional Services,24,4.666667
166,Health & Medical,33,4.625000
171,Chocolatiers & Shops,28,4.500000
140,Cupcakes,44,4.500000
82,Street Vendors,159,4.439394
143,Fruits & Veggies,42,4.375000
168,Custom Cakes,31,4.333333
182,Patisserie/Cake Shop,23,4.333333
96,Poke,106,4.258621


In [31]:
#highest rated categories if business count > 100
categories[(categories.business_count > 100)].sort_values('avg_stars', ascending = False)

,category,business_count,avg_stars
82,Street Vendors,159,4.439394
96,Poke,106,4.258621
48,Food Trucks,466,4.192708
51,Vegan,414,4.168919
89,Local Flavor,122,4.138889
98,Ramen,101,4.000000
78,Caribbean,183,4.000000
39,Vegetarian,599,3.977876
55,Middle Eastern,362,3.975610
88,Wraps,124,3.970588
